**Ejercicio 1:** Implemente un algoritmo de optimización por enjambre de partículas y utilícelo para encontrar el mínimo global de las funciones del Ejercicio 1 de la Guía de trabajos prácticos 6.
Compare los resultados en relación a los obtenidos con algoritmos genéticos, en términos de las soluciones encontradas y la velocidad de convergencia.

#### **Librerias**

In [156]:
import numpy as np
import time

#### **Algoritmo**

**Parámetros:**
- c1, c2 son pesos a la historia personal y social respecivamente, que pueden ser variables en las iteraciones como vimos en la fase de transición del SOM.
- r_1i, r_2i son vectores que tienen tantas componentes como dimensiones las particulas. Sus elementos son numeros aleatorios distribuidos entre 0 y 1 que se van aleatorizando cada vez que voy a usar la ecuación, y se aplican elemento a elemento a la resta y-x, y lo que me hace es romper el determinismo, sino llegaría a la misma posición final con todas las particulas.

**Vectores:**
- v_enjambre, cuyos elementos son la posicion(primeras columnas) y el valor(ultima columna) de cada una de las particulas
- valores, cuyos elementos son los valores que tienen las posiciones de v_enjambre
- mejores_valores, cuyos elementos son las mejores posiciones(primeras columnas) y sus valores(ultima columna) asociados que tuvo cada particula en toda su historia
- mejor_pos_global, cuyos elementos son la mejor posicion historica de todas las particulas y su valor

In [157]:
def enjambre(num_particulas, mins, maxs):
    # Crear un enjambre de partículas
    particulas = np.zeros((num_particulas, len(mins)));
    for i in range(num_particulas):
        for j in range(len(mins)):
            particulas[i][j] = np.random.uniform(mins[j], maxs[j]);
    return particulas
# particulas ahora contiene una lista de partículas con valores aleatorios en el rango especificado.      

In [158]:
def algoritmo_enjambre(num_particulas,mins,maxs,funcion_error,c,max_iteraciones,tolerancia):

    inicio = time.time()    # Inicio del contador de tiempo para ver velocidad de convergencia

    # Inicialización del enjambre:
    v_enjambre = enjambre(num_particulas,mins,maxs);
    aux_valores = np.zeros(num_particulas)
    for i in range(len(v_enjambre)):    # Valores de la función de error para cada partícula
        aux_valores[i] = funcion_error(v_enjambre[i])
    velocides = np.zeros((num_particulas,len(mins)))
    
    # Inicializo vectores para comparar y obtener mejor posición:
    mejor_pos = np.hstack((v_enjambre, aux_valores.reshape(-1, 1)))
    mejor_pos_global = np.hstack((v_enjambre[np.argmin(aux_valores)],aux_valores[np.argmin(aux_valores)]));
    
    for _ in range(max_iteraciones):

        # Para cada partícula, reviso si su valor para la función de error es el mejor personal o global.
        for i in range(num_particulas):
            resultado = funcion_error(v_enjambre[i])
            if(resultado<mejor_pos[i][-1]):             # Si partícula i es el mejor personal, reemplazo.
                mejor_pos[i][:] = np.hstack((v_enjambre[i],resultado));
                if(resultado<mejor_pos_global[-1]):     # Si partícula i es el mejor personal, me fijo si es el mejor global. Si lo es, reemplazo.
                    mejor_pos_global = np.hstack((v_enjambre[i],resultado));
        
        # Aleatorización de las r para cada partícula:
        r1 = np.random.rand(len(v_enjambre),1);
        r2 = np.random.rand(len(v_enjambre),1);

        # Calculo velocidades de manera vectorial:
        velocides = velocides + c[0]*r1*(mejor_pos[:,:-1]-v_enjambre)+c[1]*r2*(mejor_pos_global[:-1]-v_enjambre)

        # Para cada partícula, obtengo el cambio en la posición con la velocidad obtenida: 
        for i in range(num_particulas):
            suma = v_enjambre[i] + velocides[i]
            if(np.logical_and(suma > mins, suma < maxs).all()):     # Verifico que no me vaya de los límites del problema.
                v_enjambre[i] = suma
    
        # Condición de corte:
        #if(mejor_pos_global[-1]<tolerancia):
        #    print("tol")
        #    break

    fin = time.time()
    print(f'El algoritmo con {max_iteraciones} iteraciones terminó en {round(fin-inicio,4)} segundos.')

    return mejor_pos_global;
        

**Función 1:** $ f(x) = - x \sin(\sqrt{|x|}) $, con $ x \in [-512...512] $

In [159]:
def f1(x):
    return -x*np.sin(np.sqrt(abs(x)))
x = np.linspace(-512,512,100)
y_f = f1(x)
print(f'El mínimo de la función está en: [{x[np.argmin(y_f)]},{np.min(y_f)}]')

num_particulas = 50
minimos = [-512]
maximos = [512]
funcion_error = f1
c = [0.1,0.7]
max_iteraciones = 100
tolerancia = 0.2
resultado = algoritmo_enjambre(num_particulas,minimos,maximos,funcion_error,c,max_iteraciones,tolerancia);
print('El mínimo encontrado por el algoritmo de enjambres está en:',resultado)

El mínimo de la función está en: [418.909090909091,-418.4481325008988]
El algoritmo con 100 iteraciones terminó en 0.1547 segundos.
El mínimo encontrado por el algoritmo de enjambres está en: [ 420.77529273 -418.9781653 ]


**Función 2:** $ f(x, y) = (x^2 + y^2)^{0.25} [\sin^2 (50 (x^2 + y^2)^{0.1}) + 1]$, con $x, y \in [-100...100]$

In [162]:
def f2(x):
    return ((x[0]**2 + x[1]**2)**0.25)*((np.sin(50*(x[0]**2 + x[1]**2)**0.1))**2  + 1)
x = np.linspace(-100, 100, 1000)
y = np.linspace(-100, 100, 1000)
print(f'El mínimo de la función está en: [{x[np.argmin(f2([x,y]))]},{y[np.argmin(f2([x,y]))]},{np.min(f2([x,y]))}]')

num_particulas = 50;
minimos = [-100,-100];
maximos = [100,100];
funcion_error = f2;
c = [0.1,0.7];
max_iteraciones = 100;
tolerancia = 0.2
resultado = algoritmo_enjambre(num_particulas,minimos,maximos,funcion_error,c,max_iteraciones,tolerancia);
print('El mínimo encontrado por el algoritmo de enjambres está en:',resultado)

El mínimo de la función está en: [0.10010010010010717,0.10010010010010717,0.5467853625415365]
El algoritmo con 100 iteraciones terminó en 0.0667 segundos.
El mínimo encontrado por el algoritmo de enjambres está en: [-0.03270099 -0.14348862  0.46384236]
